In [2]:
import numpy as np
import pandas as pd
import os
import torch
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
import dgl
import sklearn
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.datasets import make_classification
# from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

In [15]:
X = df.iloc[:,:-1]
y = y
y = pd.Series(y)
y.index = X.index
print(str(len(X)) + ' rows')
print(str(len(X.columns)) + ' columns')


xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.33, random_state=42)
xTrainScaler = StandardScaler()
xTrain = xTrainScaler.fit_transform(xTrain)
xTrain = pd.DataFrame(xTrain,columns = X.columns)
xTest = StandardScaler().fit_transform(xTest)
xTest = pd.DataFrame(xTest,columns = X.columns)
yTrain = StandardScaler().fit_transform(yTrain.values.reshape(-1, 1))
yTest = StandardScaler().fit_transform(yTest.values.reshape(-1, 1))
yTrain = pd.Series(yTrain.flatten())
yTrain.index = xTrain.index
yTest = pd.Series(yTest.flatten())
yTest.index = xTest.index

from ControlBurn.ControlBurn import ControlBurnRegressor

cb = ControlBurnRegressor(build_forest_method = 'doublebagboost', alpha = 0.02)
cb.fit(xTrain,yTrain)

print('Number of trees grown: ' + str(len(cb.forest)))
print('Number of trees selected: ' + str(len(cb.subforest)))
print('Features selected ' + str(cb.features_selected_))


200 rows
58 columns
Number of trees grown: 77
Number of trees selected: 28
Features selected ['chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var', 'harmony_mean', 'perceptr_var', 'mfcc2_mean', 'mfcc3_mean', 'mfcc3_var', 'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean', 'mfcc7_var', 'mfcc8_mean', 'mfcc8_var', 'mfcc10_var', 'mfcc11_var', 'mfcc13_mean', 'mfcc14_mean', 'mfcc15_mean', 'mfcc16_mean', 'mfcc17_mean', 'mfcc18_var', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var']


In [3]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [4]:
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=in_feats, out_feats=hid_feats, aggregator_type='mean',feat_drop=0.01)
        self.conv2 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=hid_feats, aggregator_type='mean',feat_drop=0.01)
        self.conv3 = dglnn.SAGEConv(
            in_feats=hid_feats, out_feats=out_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.conv2(graph, h)
        h = F.relu(h)
        h = self.conv2(graph, h)
        h = F.relu(h)
        h = self.conv2(graph, h)
        h = F.relu(h)
        h = self.conv2(graph, h)
        h = F.relu(h)
        h = self.conv2(graph, h)
        h = F.relu(h)
        h = self.conv3(graph, h)
        
        return h

In [20]:
acc_cv=[]
for i in range(10):
    df=pd.read_csv('./mfcc/features_3_sec.csv')
    df=df.drop(labels="filename",axis=1)
    df=df[(df.label=='blues')|(df.label=='jazz')]
    
    
    
    df=df[['rms_var', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'mfcc5_var', 'mfcc6_mean', 'mfcc7_var', 'mfcc10_var', 'mfcc17_mean', 'mfcc20_var','label']]



    df=df.sample(n=2000)
    df.label.value_counts()
    X=StandardScaler().fit_transform(np.array(df.iloc[:,:-1],dtype=float))
    class_list=df.iloc[:,-1]
    converter=LabelEncoder()
    y=converter.fit_transform(class_list)
    print(X.shape,y.shape)
    # X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33)
    # print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

    X_train=X[0:1340,:]
    y_train=y[0:1340]
    X_test=X[1340:,:]
    y_test=y[1340:]

    print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
    mfccs_total=X
    labels_total=y
    labels_train=y_train
    labels_test=y_test

    from sklearn.decomposition import KernelPCA
    transformer = KernelPCA(n_components=2, kernel='rbf')
    kpca_mfccs_total = transformer.fit_transform(mfccs_total)
    # kpca_mfccs_total

    from sklearn.neighbors import NearestNeighbors
    num_neigh = 120
    neigh1 = NearestNeighbors(n_neighbors=num_neigh, radius=0.5)
    neigh1.fit(kpca_mfccs_total)

    adj_matrix_total = np.zeros((len(labels_total),len(labels_total)))

    for i in tqdm(range(len(labels_total))):

        dist1, nbors1 =neigh1.kneighbors([kpca_mfccs_total[i,:]])
        #only keep train neighbor
        nbors1 = nbors1[nbors1<len(labels_train)]

        neigh_total = np.array(nbors1).squeeze().flatten()
        neigh_total = np.unique(neigh_total)
        adj_matrix_total[i,neigh_total] = 1
        adj_matrix_total[neigh_total,i] = 1


     #train graph
    nx_graph_train = nx.from_numpy_matrix(adj_matrix_total[0:len(labels_train),0:len(labels_train)])
    g_dgl_euclid_train = dgl.from_networkx(nx_graph_train)
    g_dgl_euclid_train

    #test graph
    nx_graph_test = nx.from_numpy_matrix(adj_matrix_total[len(labels_train):,len(labels_train):])
    g_dgl_euclid_test = dgl.from_networkx(nx_graph_test)
    g_dgl_euclid_test
    #Total Graph
    nx_graph_total= nx.from_numpy_matrix(adj_matrix_total)
    g_dgl_euclid_total = dgl.from_networkx(nx_graph_total)
    g_dgl_euclid_total

    ###create train and valid

    train_mask = np.full(len(labels_train),True)
    t_mask2 = th.BoolTensor(train_mask)

    test_mask = np.full(len(labels_test),True)
    t_mask_test = th.BoolTensor(test_mask)


    inputs_g_train = torch.tensor(kpca_mfccs_total[:len(labels_train)]).float()
    labels_graph_train = torch.tensor(labels_train).long()

    inputs_g_test = torch.tensor(kpca_mfccs_total[len(labels_train):]).float()
    labels_graph_test = torch.tensor(labels_test).long()

    train_mask_total = np.full(len(labels_total),True)
    t_mask_total = th.BoolTensor(train_mask_total)

    model_gsage = SAGE(in_feats=2, hid_feats=100, out_feats=2)
    opt = th.optim.Adam(model_gsage.parameters(),lr=1e-3)
    model_gsage.train()

    for epoch in range(500):
        out = model_gsage(g_dgl_euclid_train, inputs_g_train)
        # compute loss
        loss = F.cross_entropy(out, labels_graph_train)
        # compute validation accuracy
        acc = evaluate(model_gsage, g_dgl_euclid_train, inputs_g_train, labels_graph_train, t_mask2)

        opt.zero_grad()
        loss.backward()
        opt.step()
        print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f}".format(
                epoch, loss.item(), acc))

    acc_cv.append(evaluate(model_gsage, g_dgl_euclid_test, inputs_g_test, labels_graph_test, t_mask_test))


(2000, 10) (2000,)
(1340, 10) (660, 10) (1340,) (660,)


100%|██████████| 2000/2000 [00:00<00:00, 2688.44it/s]


Epoch 00000 | Loss 1.1531 | Test Acc 0.4993
Epoch 00001 | Loss 0.7663 | Test Acc 0.4993
Epoch 00002 | Loss 0.5965 | Test Acc 0.6463
Epoch 00003 | Loss 0.5176 | Test Acc 0.8507
Epoch 00004 | Loss 0.4753 | Test Acc 0.8582
Epoch 00005 | Loss 0.4408 | Test Acc 0.8627
Epoch 00006 | Loss 0.4143 | Test Acc 0.8612
Epoch 00007 | Loss 0.3930 | Test Acc 0.8664
Epoch 00008 | Loss 0.3756 | Test Acc 0.8694
Epoch 00009 | Loss 0.3616 | Test Acc 0.8694
Epoch 00010 | Loss 0.3504 | Test Acc 0.8679
Epoch 00011 | Loss 0.3407 | Test Acc 0.8709
Epoch 00012 | Loss 0.3317 | Test Acc 0.8739
Epoch 00013 | Loss 0.3236 | Test Acc 0.8769
Epoch 00014 | Loss 0.3165 | Test Acc 0.8746
Epoch 00015 | Loss 0.3110 | Test Acc 0.8799
Epoch 00016 | Loss 0.3064 | Test Acc 0.8806
Epoch 00017 | Loss 0.3027 | Test Acc 0.8791
Epoch 00018 | Loss 0.2997 | Test Acc 0.8836
Epoch 00019 | Loss 0.2972 | Test Acc 0.8851
Epoch 00020 | Loss 0.2954 | Test Acc 0.8828
Epoch 00021 | Loss 0.2945 | Test Acc 0.8843
Epoch 00022 | Loss 0.2946 | Test

Epoch 00187 | Loss 0.2796 | Test Acc 0.8836
Epoch 00188 | Loss 0.2795 | Test Acc 0.8843
Epoch 00189 | Loss 0.2794 | Test Acc 0.8843
Epoch 00190 | Loss 0.2794 | Test Acc 0.8843
Epoch 00191 | Loss 0.2793 | Test Acc 0.8851
Epoch 00192 | Loss 0.2793 | Test Acc 0.8843
Epoch 00193 | Loss 0.2792 | Test Acc 0.8851
Epoch 00194 | Loss 0.2791 | Test Acc 0.8851
Epoch 00195 | Loss 0.2791 | Test Acc 0.8851
Epoch 00196 | Loss 0.2790 | Test Acc 0.8851
Epoch 00197 | Loss 0.2789 | Test Acc 0.8851
Epoch 00198 | Loss 0.2789 | Test Acc 0.8851
Epoch 00199 | Loss 0.2788 | Test Acc 0.8851
Epoch 00200 | Loss 0.2788 | Test Acc 0.8851
Epoch 00201 | Loss 0.2787 | Test Acc 0.8851
Epoch 00202 | Loss 0.2786 | Test Acc 0.8851
Epoch 00203 | Loss 0.2786 | Test Acc 0.8851
Epoch 00204 | Loss 0.2785 | Test Acc 0.8851
Epoch 00205 | Loss 0.2784 | Test Acc 0.8851
Epoch 00206 | Loss 0.2784 | Test Acc 0.8851
Epoch 00207 | Loss 0.2783 | Test Acc 0.8851
Epoch 00208 | Loss 0.2782 | Test Acc 0.8851
Epoch 00209 | Loss 0.2782 | Test

Epoch 00374 | Loss 0.2628 | Test Acc 0.8896
Epoch 00375 | Loss 0.2626 | Test Acc 0.8903
Epoch 00376 | Loss 0.2620 | Test Acc 0.8903
Epoch 00377 | Loss 0.2615 | Test Acc 0.8918
Epoch 00378 | Loss 0.2613 | Test Acc 0.8903
Epoch 00379 | Loss 0.2613 | Test Acc 0.8903
Epoch 00380 | Loss 0.2614 | Test Acc 0.8925
Epoch 00381 | Loss 0.2617 | Test Acc 0.8903
Epoch 00382 | Loss 0.2623 | Test Acc 0.8896
Epoch 00383 | Loss 0.2625 | Test Acc 0.8910
Epoch 00384 | Loss 0.2626 | Test Acc 0.8910
Epoch 00385 | Loss 0.2618 | Test Acc 0.8903
Epoch 00386 | Loss 0.2608 | Test Acc 0.8933
Epoch 00387 | Loss 0.2603 | Test Acc 0.8918
Epoch 00388 | Loss 0.2603 | Test Acc 0.8910
Epoch 00389 | Loss 0.2608 | Test Acc 0.8918
Epoch 00390 | Loss 0.2614 | Test Acc 0.8918
Epoch 00391 | Loss 0.2621 | Test Acc 0.8918
Epoch 00392 | Loss 0.2616 | Test Acc 0.8903
Epoch 00393 | Loss 0.2611 | Test Acc 0.8918
Epoch 00394 | Loss 0.2599 | Test Acc 0.8918
Epoch 00395 | Loss 0.2594 | Test Acc 0.8918
Epoch 00396 | Loss 0.2596 | Test

...

Epoch 00379 | Loss 0.2972 | Test Acc 0.8746
Epoch 00380 | Loss 0.2971 | Test Acc 0.8746
Epoch 00381 | Loss 0.2971 | Test Acc 0.8746
Epoch 00382 | Loss 0.2970 | Test Acc 0.8746
Epoch 00383 | Loss 0.2970 | Test Acc 0.8746
Epoch 00384 | Loss 0.2969 | Test Acc 0.8746
Epoch 00385 | Loss 0.2969 | Test Acc 0.8746
Epoch 00386 | Loss 0.2969 | Test Acc 0.8746
Epoch 00387 | Loss 0.2968 | Test Acc 0.8746
Epoch 00388 | Loss 0.2968 | Test Acc 0.8746
Epoch 00389 | Loss 0.2967 | Test Acc 0.8746
Epoch 00390 | Loss 0.2967 | Test Acc 0.8746
Epoch 00391 | Loss 0.2966 | Test Acc 0.8746
Epoch 00392 | Loss 0.2966 | Test Acc 0.8754
Epoch 00393 | Loss 0.2965 | Test Acc 0.8754
Epoch 00394 | Loss 0.2965 | Test Acc 0.8746
Epoch 00395 | Loss 0.2965 | Test Acc 0.8754
Epoch 00396 | Loss 0.2964 | Test Acc 0.8754
Epoch 00397 | Loss 0.2964 | Test Acc 0.8754
Epoch 00398 | Loss 0.2963 | Test Acc 0.8754
Epoch 00399 | Loss 0.2963 | Test Acc 0.8754
Epoch 00400 | Loss 0.2962 | Test Acc 0.8746
Epoch 00401 | Loss 0.2962 | Test

100%|██████████| 2000/2000 [00:00<00:00, 2845.05it/s]


Epoch 00000 | Loss 0.8147 | Test Acc 0.5261
Epoch 00001 | Loss 0.4914 | Test Acc 0.8209
Epoch 00002 | Loss 0.3867 | Test Acc 0.8463
Epoch 00003 | Loss 0.3436 | Test Acc 0.8709
Epoch 00004 | Loss 0.3277 | Test Acc 0.8634
Epoch 00005 | Loss 0.3225 | Test Acc 0.8649
Epoch 00006 | Loss 0.3174 | Test Acc 0.8739
Epoch 00007 | Loss 0.3166 | Test Acc 0.8813
Epoch 00008 | Loss 0.3183 | Test Acc 0.8806
Epoch 00009 | Loss 0.3177 | Test Acc 0.8828
Epoch 00010 | Loss 0.3157 | Test Acc 0.8843
Epoch 00011 | Loss 0.3138 | Test Acc 0.8806
Epoch 00012 | Loss 0.3104 | Test Acc 0.8828
Epoch 00013 | Loss 0.3068 | Test Acc 0.8836
Epoch 00014 | Loss 0.3045 | Test Acc 0.8843
Epoch 00015 | Loss 0.3034 | Test Acc 0.8821
Epoch 00016 | Loss 0.3028 | Test Acc 0.8806
Epoch 00017 | Loss 0.3028 | Test Acc 0.8799
Epoch 00018 | Loss 0.3028 | Test Acc 0.8791
Epoch 00019 | Loss 0.3019 | Test Acc 0.8813
Epoch 00020 | Loss 0.3007 | Test Acc 0.8828
Epoch 00021 | Loss 0.2995 | Test Acc 0.8828
Epoch 00022 | Loss 0.2983 | Test

Epoch 00190 | Loss 0.2813 | Test Acc 0.8836
Epoch 00191 | Loss 0.2813 | Test Acc 0.8843
Epoch 00192 | Loss 0.2813 | Test Acc 0.8836
Epoch 00193 | Loss 0.2815 | Test Acc 0.8843
Epoch 00194 | Loss 0.2816 | Test Acc 0.8851
Epoch 00195 | Loss 0.2816 | Test Acc 0.8851
Epoch 00196 | Loss 0.2816 | Test Acc 0.8843
Epoch 00197 | Loss 0.2812 | Test Acc 0.8851
Epoch 00198 | Loss 0.2809 | Test Acc 0.8843
Epoch 00199 | Loss 0.2808 | Test Acc 0.8836
Epoch 00200 | Loss 0.2810 | Test Acc 0.8851
Epoch 00201 | Loss 0.2814 | Test Acc 0.8843
Epoch 00202 | Loss 0.2815 | Test Acc 0.8843
Epoch 00203 | Loss 0.2813 | Test Acc 0.8843
Epoch 00204 | Loss 0.2808 | Test Acc 0.8851
Epoch 00205 | Loss 0.2805 | Test Acc 0.8836
Epoch 00206 | Loss 0.2807 | Test Acc 0.8836
Epoch 00207 | Loss 0.2810 | Test Acc 0.8851
Epoch 00208 | Loss 0.2811 | Test Acc 0.8843
Epoch 00209 | Loss 0.2809 | Test Acc 0.8843
Epoch 00210 | Loss 0.2805 | Test Acc 0.8836
Epoch 00211 | Loss 0.2801 | Test Acc 0.8851
Epoch 00212 | Loss 0.2802 | Test

Epoch 00379 | Loss 0.2654 | Test Acc 0.8903
Epoch 00380 | Loss 0.2654 | Test Acc 0.8881
Epoch 00381 | Loss 0.2657 | Test Acc 0.8888
Epoch 00382 | Loss 0.2659 | Test Acc 0.8866
Epoch 00383 | Loss 0.2663 | Test Acc 0.8888
Epoch 00384 | Loss 0.2663 | Test Acc 0.8896
Epoch 00385 | Loss 0.2662 | Test Acc 0.8896
Epoch 00386 | Loss 0.2654 | Test Acc 0.8866
Epoch 00387 | Loss 0.2646 | Test Acc 0.8903
Epoch 00388 | Loss 0.2640 | Test Acc 0.8888
Epoch 00389 | Loss 0.2636 | Test Acc 0.8873
Epoch 00390 | Loss 0.2637 | Test Acc 0.8910
Epoch 00391 | Loss 0.2640 | Test Acc 0.8888
Epoch 00392 | Loss 0.2647 | Test Acc 0.8881
Epoch 00393 | Loss 0.2651 | Test Acc 0.8896
Epoch 00394 | Loss 0.2656 | Test Acc 0.8918
Epoch 00395 | Loss 0.2650 | Test Acc 0.8873
Epoch 00396 | Loss 0.2640 | Test Acc 0.8903
Epoch 00397 | Loss 0.2629 | Test Acc 0.8873
Epoch 00398 | Loss 0.2625 | Test Acc 0.8881
Epoch 00399 | Loss 0.2625 | Test Acc 0.8896
Epoch 00400 | Loss 0.2631 | Test Acc 0.8866
Epoch 00401 | Loss 0.2638 | Test

In [21]:
np.mean(acc_cv)

0.6557575757575758

In [22]:
acc_cv

[0.5060606060606061,
 0.8378787878787879,
 0.603030303030303,
 0.5969696969696969,
 0.5075757575757576,
 0.5196969696969697,
 0.7924242424242425,
 0.85,
 0.5015151515151515,
 0.8424242424242424]